# Analytics Zoo Neural Network : Prosper Loan Dataset

We are going to look at the prosper loan dataset.  This dataset shows a history of loans made by Prosper.

The idea with this dataset is to try to predict whether a loan will be "good" or "bad." There are many ways a loan can be bad, from late payments to defaults, but we will turn this into a binary choice between "good" loans and "bad" loans.

We will use a neural network in the Keras-style API in Analytics Zoo to do the prediction of "good" loans or "bad.

This uses Analytics Zoo.   If you want to see a "vanilla" Tensorflow version of this lab, click [here](./prosper-tf.ipynb).

In [ ]:
from zoo.common.nncontext import init_nncontext
sc = init_nncontext("Prosper Classification")  # Initialize the 

## Load the Data

We are going to load the data of the prosper loan dataset.

Notice we are first loading this into a Pandas dataframe. This is fine for a small dataset, but we will need more than this for a large "at scale" notebook.

Notice the very large numbers of columns. We are going to take only a selection of those columns.

In [ ]:
import pandas as pd

## small file, start with this
#datafile = "https://s3.amazonaws.com/elephantscale-public/data/prosper-loan/prosper-loan-data-sample.csv"
## this is a large file
datafile = "https://s3.amazonaws.com/elephantscale-public/data/prosper-loan/prosper-loan-data.csv.gz"

data = pd.read_csv(datafile)
data.tail()

## Look at some summary data

Let's look at some summary data here.

We'd like to see the cardinality of the categorical columns.

In [ ]:
print(data['LoanStatus'].value_counts())


It looks like "good" loans outnumber bad about 2-1.  That's a class imbalance but not a dramatic one. We can probably get away with using it as is.

Let's now look at the Employment Status:

In [ ]:

print(data['EmploymentStatus'].value_counts())



It appears that there are a variety of different statuses although Full-Time and Employed (which seem to be the same thing), are the majority.

## Converting Categorical columns 

Convert categorical columns to numeric.   
Here let's convert **EmploymentStatus** column

In [ ]:
# use pd.factorize on EmploymentStatus, ListingCategory

data['EmploymentStatusFactor'] = pd.factorize(data['EmploymentStatus'])[0]
data['ListingCategoryFactor'] = pd.factorize(data['ListingCategory'])[0]

## Build feature vectors 

We're going to use this from the `feature_columns` list.  This will get *only* the feature columns.

In [ ]:
feature_columns = ['EmploymentStatusFactor', 'CreditScore', 'StatedMonthlyIncome', 'ListingCategoryFactor']
features = data[feature_columns]
label = data['LoanStatus']
features.tail()

## Split Data into training and test.

We will split our the data up into training and test. Here we do 70% training, 30% test.

In [ ]:
## Split the data into 70% training and 30% test sets 

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, label)
print("training set = " , len(train_x))  # Show number of rows intraining set
pd.DataFrame(train_x).tail()  # Show last few ros in training set.

In [ ]:
print("test set = " , len(test_x))  # Show number of rows in test sete
pd.DataFrame(test_x).tail()   # Show last few rows of test set

In [ ]:
print(train_x.keys().tolist())  # Print out the names of the columns

## Neural Network

Here we are going to define our network. We will do this with the Analytics Zoo Keras style API.

Here's what we have:

 * Input Layer
 * Hidden Layer (4 Neurons), with Dropout
 * Hidden Layer (4 Neurons), with Dropout
 * Output Layer (1 Neuron)
 
This is *binary* clasification.
 
We will train with Adam optimizer, which is a good all-around optimizer that doesn't need us to play much with hyperparameters.

In [ ]:
from zoo.pipeline.api.keras.layers import Dense, Dropout
from zoo.pipeline.api.keras.models import Sequential

def build_model(train_x):
  model = Sequential()

  model.add(Dense(4, input_dim=len(train_x.columns), activation='tanh'))
  model.add(Dropout(0.2))
  model.add(Dense(4, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(Dense(output_dim=1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model


model = build_model(train_x)
model.summary()

Now it's time to train. We will do this with the `model.fit()` from analytics zoo. Notice the Keras-like `.fit` semantics.

In [ ]:
%%time
# Train the model
print("Training begins.")
model.fit(
    train_x.values,
    train_y.values,
    batch_size=500,
    nb_epoch=2)
print("Training completed.")

In [ ]:
predictions = model.predict(test_x.values)

## Evaluate the model.

Let us check to see how the model did, using accuracy as a measure.

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
accuracy_score(test_y, np.rint(np.concatenate(predictions.collect())))

The results indicate that we get 67% accuracy, meaning we are right 2/3 of the time.

##  Improve Accuracy

### Add more features
Look at the schema of the full dataset.  Are there any columns you want to add? Make sure you up the number of neurons in the hidden layer as you add more features.

## Conclusion

1. Models can be used for both Classification and regression
2. Classification ususally uses a loss function of cross-entropy rather than MSE.
3. A Multilayer Peceptron network can be built with Keras style `Dense` layers.

